In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
import re
from tqdm import tqdm
import pandas as pd
import requests

In [13]:
url = 'https://nyheder.tv2.dk/seneste'
driver = webdriver.Edge()
driver.get(url)

In [14]:
cookie = driver.find_element(By.CSS_SELECTOR, '#onetrust-reject-all-handler')
cookie.click()

In [11]:
Vis_flere = driver.find_element(By.CSS_SELECTOR, '.js-tc_load-more__bound')
Vis_flere.click()

In [161]:
"""
for i in range(5):
    Vis_flere = driver.find_element(By.CSS_SELECTOR, '.js-tc_load-more__bound')
    Vis_flere.click()
    time.sleep(5)
"""

"\nfor i in range(5):\n    Vis_flere = driver.find_element(By.CSS_SELECTOR, '.js-tc_load-more__bound')\n    Vis_flere.click()\n    time.sleep(5)\n"

In [17]:
html = driver.page_source
soup = BeautifulSoup(html, 'lxml')

Headline = []
Category = []
Authors = []
Time = []
All_text = []
nummer=0

max_retries = 10
retries = 0

for i in tqdm(range(5)):
    retries = 0
    while retries < max_retries:
        try:
            one_column_lists = soup.find_all('ul', {'data-columns': '1'})[i]
            break
        except:
            retries = retries + 1
            time.sleep(1)
    
    if retries == max_retries:
        print(f"Max tries at: Number {nummer} and i: {i}")
    
    for one_column_list in one_column_lists:
        artikler = one_column_list.find_all(class_='tc_teaser')

        for artikel in artikler:
            nummer = nummer + 1

            Category.append(artikel.find('span', class_='tc_label tc_label--color-nyheder').text)
            Headline.append(artikel.find('a', class_='tc_teaser__link').get('aria-label'))

            href = artikel.find('a', class_='tc_teaser__link').get('href') #if link else None

            text = []
            if href:
                driver.execute_script(f'window.open("{href}", "_blank");')
                driver.switch_to.window(driver.window_handles[-1])

                new_html = driver.page_source
                new_soup = BeautifulSoup(new_html, 'lxml')

                #Category.append(new_soup.find('div', class_='tc_page__label').find('a', class_='tc_label tc_label--color-nyheder').text)
                #Headline.append(new_soup.find('header', class_="tc_page__header").find('h1').text)

                Authors.append(new_soup.find('div', class_="tc_byline").find('strong').text)
                Time.append(new_soup.find('time', class_='tc_timestamp').get('datetime'))
                
                if new_soup.find('div', class_="tc_liveblog__body", attrs={'data-adobe-context':"article-body"}):
                    article_body_div = new_soup.find('div', class_="tc_liveblog__body", attrs={'data-adobe-context':"article-body"})
                else:
                    article_body_div = new_soup.find('div', class_='tc_page__body', attrs={'data-adobe-context': 'article-body'})
                
                if article_body_div:
                    rich_content_paragraphs = article_body_div.find_all('p')

                    for paragraph in rich_content_paragraphs:
                        text.append(paragraph.text)

                All_text.append(text)

                driver.close()
                driver.switch_to.window(driver.window_handles[0])

            #print(nummer, Category[nummer-1], ':', Headline[nummer-1])
    Vis_flere = driver.find_element(By.CSS_SELECTOR, '.js-tc_load-more__bound')
    Vis_flere.click()
    #print(i)

 60%|██████████████████████████████████████████████████▍                                 | 3/5 [00:26<00:19,  9.89s/it]

Max tries at: Number 29 and i: 3


 80%|███████████████████████████████████████████████████████████████████▏                | 4/5 [00:49<00:15, 15.36s/it]

Max tries at: Number 48 and i: 4


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [01:13<00:00, 14.72s/it]


In [20]:
print(Category[0])
print(Authors[0])
print(Headline[0])
print(Time[0])
print(All_text[0])

Politik
Rikke Struck Westersø
SF holder sommergruppemøde
2023-08-14T08:44:20.010Z
['Hvad sker der? SF holder sommergruppemøde ved HOFORs drikkevandsboring i Ringsted. Der er pressemøde klokken 11. ', 'Hvad skal det handle om? Danmark er ikke længere et grønt foregangsland, fordi der tøves i klimapolitikken, mener SF. Partiet kommer derfor i forbindelse med deres sommergruppemøder med en række klimaforslag samt et krav til regeringen om akut politisk klimahandling.', 'Nogle af partiets spidser er nu dukket op til pressemødet, der skal gå i gang lige om lidt.', 'Det er SF-formand Pia Olsen Dyhr, partiets politiske ordfører Signe Munk og Karina Lorentzen Dehnhardt, der er retsordfører.\xa0', 'SF står et rigtig godt sted lige nu. De er med afstand Danmarks næststørste parti i målingerne.', 'Det er i høj grad udtryk for, at de har været i stand til at tiltrække rigtig mange skuffede socialdemokratiske vælgere. De er på mange måder det naturlige andetvalg for mange vælgere – også Enhedsliste

In [21]:
data = {
    'Category': Category,
    'Authors': Authors,
    'Headline': Headline,
    'Time': Time,
    'All_text': All_text
}

df = pd.DataFrame(data)
df

,Category,Authors,Headline,Time,All_text
0,Politik,Rikke Struck Westersø,SF holder sommergruppemøde,2023-08-14T08:44:20.010Z,[Hvad sker der? SF holder sommergruppemøde ved...
1,Lokalt,Marie Kjempff,Dyreejere vågnede op til makabert syn: - Det h...,2023-08-14T08:42:03.887Z,[Ubudne gæster i Dragør er stukket af med høns...
2,Lokalt,Iben Sørensen,Skolepatruljer chikaneres i trafikken,2023-08-14T08:34:59.764Z,[Mandag morgen var politiet på plads ved Rundh...
3,Politik,Rikke Steenbech,Hård kritik af regeringens klimapolitik: - Det...,2023-08-14T08:28:29.233Z,"[I SF mener de ikke, at regeringen er ambitiøs..."
4,Lokalt,Mathilde Justesen Ranners,Lokale redder købmandsbutik med millioner - og...,2023-08-14T08:19:53.601Z,[Flere steder i Danmark reddes pressede købmæn...
...,...,...,...,...,...
62,Politik,Jeppe Tholstrup Bach,Regeringen i bakgear – dropper idé om egen ops...,2023-08-13T15:27:07.094Z,[Regeringen har ikke planer om at fremsætte et...
63,Samfund,Ritzau,Lodsejer afbryder traditionsrigt racerløb for ...,2023-08-13T15:16:28.122Z,[En lodsejer valgte at blokere strækningen til...
64,Udland,Ritzau,Storbritannien advarer borgere mod risiko for ...,2023-08-13T14:23:33.700Z,"[Det er meget sandsynligt, at terrorister vil ..."
65,Udland,Lars Bach Jørgensen,Nudister klager over turister med badetøj,2023-08-13T14:00:12.787Z,[Ved Costa Brava i Spanien efterlyser naturist...


In [128]:
#df.to_csv('Data.csv', index=False)